In [2]:
#Attempt 1
import geopandas as gpd
from shapely.geometry import Point

# Load zone boundary data
zones = gpd.read_file(r"C:\Users\NettajiBabuji\Downloads\taxi_zones.json")

# Assuming coordinates is a list of (latitude, longitude) tuples
coordinates = [(40.7128, -74.0060), (40.7484, -73.9857), ...]

# Convert coordinates to Points
points = [Point(lon, lat) for lat, lon in coordinates]

# Create a GeoDataFrame from the points
points_gdf = gpd.GeoDataFrame(geometry=points, crs=zones.crs)

# Perform spatial join to find which zone each point falls within
joined = gpd.sjoin(points_gdf, zones, how="left", op="within")

# Extract zone IDs
zone_ids = joined['zone_id']

# Output results
for coordinate, zone_id in zip(coordinates, zone_ids):
    print(f"Coordinate {coordinate} is in zone ID {zone_id}")


TypeError: cannot unpack non-iterable ellipsis object

In [4]:
## Tyler's attempt
 
 
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Proj, transform
 
 
# Load the GeoJSON file containing multipolygon geometries
nyc_zones_in = gpd.read_file(r'C:\Users\NettajiBabuji\Downloads\taxi_zones.json')
 
# Reproject the GeoDataFrame to EPSG:4326 (latitude-longitude)
source_crs = {'init': 'epsg:2263'}  # New York State Plane Coordinate System
target_crs = {'init': 'epsg:4326'}  # WGS84 (latitude/longitude)
 
nyc_zones = nyc_zones_in.to_crs(epsg=4326)
 
 
# Define the projection for LCC (replace these values with your specific projection parameters)
lcc_proj = Proj(proj='lcc',  # Lambert Conformal Conic
                       datum='WGS84',  # WGS84 datum
                       ellps='WGS84',
                       lat_1=40.666667, lon_1=-74,  # First standard parallel and longitude
                       lat_2=41.033333, lon_2=-74,  # Second standard parallel and longitude
                       lat_0=40.166667, lon_0=-74,
                       units='us-ft')  # Latitude and longitude of origin
 
wgs84_projection = Proj(proj='latlong', datum='WGS84', ellps='WGS84', no_defs=True)
 
 
#display(nyc_zones)
#print(nyc_zones.type)
 
# # Define a function to extract latitude and longitude from a geometry
def extract_lon_lat(row):
    # print(row)
 
    if row.type == 'MultiPolygon':
 
        lon_lat_list = []
 
        # Iterate over each polygon in the MultiPolygon
        for polygon in nyc_zones.loc[1]['geometry'].geoms:
            # Access exterior coordinates of the polygon
            x_coords, y_coords = polygon.exterior.coords.xy
           
            # Convert the exterior coordinates to latitude and longitude using the projection
            lon_lat_list.extend([(x, y) for x, y in zip(x_coords, y_coords)])
 
        return [transform(lcc_proj, wgs84_projection, x, y) for x, y in lon_lat_list]
    else:
        lon, lat = row.exterior.coords.xy
        return [transform(lcc_proj, wgs84_projection, x, y) for x, y in zip(lon, lat)]
 
# # Apply the function to the geometry column to get latitudes and longitudes
#nyc_zones['lon_lat'] = nyc_zones.apply(extract_lon_lat)
 
test = nyc_zones.loc[1:4]['geometry'].apply(extract_lon_lat)
display(test)
 
# nyc_zones['lon_lat'] = nyc_zones['geometry'].apply(extract_lon_lat)
# display(nyc_zones)

C:\Users\NettajiBabuji\AppData\Local\Temp\ipykernel_28056\4169500696.py:38: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if row.type == 'MultiPolygon':
C:\Users\NettajiBabuji\AppData\Local\Temp\ipykernel_28056\4169500696.py:50: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return [transform(lcc_proj, wgs84_projection, x, y) for x, y in lon_lat_list]
C:\Users\NettajiBabuji\AppData\Local\Temp\ipykernel_28056\4169500696.py:53: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return [transform(lcc_proj, wgs84_projection, x, y) for x, y in zip(lon, lat)]


1    [(-70.27479911863823, 40.55467052911126), (-70...
2    [(-70.28598397073505, 40.80876781854231), (-70...
3    [(-70.42597765742205, 40.66333200122877), (-70...
4    [(-70.63160731859519, 40.5124118784895), (-70....
Name: geometry, dtype: object